In [1]:
import json
import folium
import datetime
import requests
import numpy as np
import pandas as pd
import geopandas as gpd
from bs4 import BeautifulSoup
from geopandas.tools import sjoin
from geopy.geocoders import Nominatim
from shapely.geometry import Point, Polygon

## Dealing with GeoJson

In [2]:
geo_df = gpd.read_file("data/chicago_boundaries.geojson")
geo_df.sort_values("pri_neigh").head(10)

,pri_neigh,sec_neigh,shape_area,shape_len,geometry
52,Albany Park,"NORTH PARK,ALBANY PARK",53542230.819,39339.016439,(POLYGON ((-87.70403771340104 41.9735515838182...
42,Andersonville,ANDERSONVILLE,9584592.89906,12534.092625,(POLYGON ((-87.66114249176968 41.9763032707800...
78,Archer Heights,"ARCHER HEIGHTS,WEST ELSDON",55922505.7212,31880.02103,(POLYGON ((-87.71436934735939 41.8260405636423...
8,Armour Square,"ARMOUR SQUARE,CHINATOWN",17141468.6356,24359.189625,(POLYGON ((-87.62920071904188 41.8471270613852...
21,Ashburn,ASHBURN,135460337.208,54818.154632,(POLYGON ((-87.71254775561138 41.7573373338274...
70,Auburn Gresham,AUBURN GRESHAM,105065353.602,46757.721716,(POLYGON ((-87.63990005778429 41.7561457832194...
94,Austin,AUSTIN,170037750.826,55473.345911,(POLYGON ((-87.75619515209075 41.9154695475812...
9,Avalon Park,"AVALON PARK,CALUMET HEIGHTS",34852737.7366,27630.822534,(POLYGON ((-87.58565529833413 41.7515019433031...
12,Avondale,"IRVING PARK,AVONDALE",55290595.482,34261.933404,(POLYGON ((-87.68798678784357 41.9361039411813...
93,Belmont Cragin,"BELMONT CRAGIN,HERMOSA",109099407.211,43311.706886,(POLYGON ((-87.74142999502229 41.9169848303604...


In [3]:
neighs = geo_df.pri_neigh.tolist()

In [4]:
def plot_map(geo_df, data=None, column_name=None, map_name="map.html"):
    m = folium.Map(location=[41.8755616, -87.6244212], tiles='Mapbox Bright', zoom_start=11) ## This location is Chicago
    
    if data is None:
        folium.GeoJson(
            geo_df,
            name='geojson'
        ).add_to(m)
    else:
        bins = list(data.quantile(np.linspace(0, 1, 15))) ## to use bins, gotta check fill_color
        folium.Choropleth(
            geo_data=geo_df,
            name='choropleth',
            data=data,
            columns=["pri_neigh", column_name],
            key_on='feature.properties.pri_neigh',
            fill_color='YlGnBu',
            fill_opacity=0.7,
            line_opacity=0.2
            #bins=bins
        ).add_to(m)
        
    m.save(map_name)

In [6]:
plot_map(geo_df)

## Dealing with Food Inspections

In [7]:
inspections = pd.read_csv("data/food-inspections.csv")
inspections = inspections.dropna(subset=["Latitude", "Longitude"]) ## we drop if we do not know the location
inspections["date"] = pd.to_datetime(inspections["Inspection Date"])
inspections.columns

Index(['Inspection ID', 'DBA Name', 'AKA Name', 'License #', 'Facility Type',
       'Risk', 'Address', 'City', 'State', 'Zip', 'Inspection Date',
       'Inspection Type', 'Results', 'Violations', 'Latitude', 'Longitude',
       'Location', 'Historical Wards 2003-2015', 'Zip Codes',
       'Community Areas', 'Census Tracts', 'Wards', 'date'],
      dtype='object')

In [8]:
inspections.head()

,Inspection ID,DBA Name,AKA Name,License #,Facility Type,Risk,Address,City,State,Zip,...,Violations,Latitude,Longitude,Location,Historical Wards 2003-2015,Zip Codes,Community Areas,Census Tracts,Wards,date
0,2320315,SERENDIPITY CHILDCARE,SERENDIPITY CHILDCARE,2216009.0,Daycare Above and Under 2 Years,Risk 1 (High),1300 W 99TH ST,CHICAGO,IL,60643.0,...,NaN,41.714168,-87.655291,"{'longitude': '41.7141680989703', 'latitude': ...",NaN,NaN,NaN,NaN,NaN,2019-10-23
1,2320342,YOLK TEST KITCHEN,YOLK TEST KITCHEN,2589655.0,Restaurant,Risk 1 (High),1767 N MILWAUKEE AVE,CHICAGO,IL,60647.0,...,23. PROPER DATE MARKING AND DISPOSITION - Comm...,41.913588,-87.682203,"{'longitude': '41.9135877900482', 'latitude': ...",NaN,NaN,NaN,NaN,NaN,2019-10-23
2,2320328,LAS ASADAS MEXICAN GRILL,LAS ASADAS MEXICAN GRILL,2583309.0,Restaurant,Risk 1 (High),3834 W 47TH ST,CHICAGO,IL,60632.0,...,NaN,41.808025,-87.720037,"{'longitude': '41.80802515275297', 'latitude':...",NaN,NaN,NaN,NaN,NaN,2019-10-23
3,2320319,LA PALAPITA,LA PALAPITA,2694702.0,Restaurant,Risk 1 (High),3834 W 47TH ST,CHICAGO,IL,60632.0,...,47. FOOD & NON-FOOD CONTACT SURFACES CLEANABLE...,41.808025,-87.720037,"{'longitude': '41.80802515275297', 'latitude':...",NaN,NaN,NaN,NaN,NaN,2019-10-23
4,2320228,47TH ST CANTINA,47TH ST CANTINA,2678250.0,Liquor,Risk 3 (Low),4311 W 47TH ST,CHICAGO,IL,60632.0,...,"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...",41.807662,-87.731480,"{'longitude': '41.80766199360051', 'latitude':...",NaN,NaN,NaN,NaN,NaN,2019-10-22


In [5]:
def get_nearest_neigh(point, geo_df):
    idx = geo_df.geometry.distance(point).idxmin()
    return geo_df.loc[idx, 'pri_neigh']

## key: key that will be used to join
def assign_neigh(geo_df, data, key, latitude="Latitude", longitude="Longitude", verbose=False):
    geometry = [Point(x, y) for x, y in zip(data[longitude], data[latitude])]
    crs = {'init': 'epsg:4326'}
    data_to_join = gpd.GeoDataFrame(data[[key, latitude, longitude]], 
                                       crs=crs,
                                       geometry=geometry)
    points_to_neigh = sjoin(data_to_join, geo_df, how='left')
    
    neigh_not_found = points_to_neigh[pd.isna(points_to_neigh.pri_neigh)]
    
    if verbose:
        print("There are {} points without an exact neighborhood".format(len(neigh_not_found)))
    
    neigh_not_found.pri_neigh = neigh_not_found.copy().geometry.apply(get_nearest_neigh, geo_df=geo_df)
    points_to_neigh.loc[neigh_not_found.index] = neigh_not_found
    
    if verbose:
        print("There are {} points without an exact neighborhood"\
              .format(len(points_to_neigh[pd.isna(points_to_neigh.pri_neigh)])))
    
    return points_to_neigh[[key, "pri_neigh", "sec_neigh"]]
    

In [10]:
inspection_location = assign_neigh(geo_df=geo_df, data=inspections, key="Inspection ID", verbose=True)

There are 2819 points without an exact neighborhood


C:\Users\Gabriel\Miniconda3\envs\ada\lib\site-packages\pandas\core\generic.py:5208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


There are 0 points without an exact neighborhood


In [11]:
inspections = inspections.merge(inspection_location, on="Inspection ID")

In [12]:
neigh_inspections = inspections.groupby("pri_neigh").count()["Inspection ID"]
plot_map(geo_df, data=neigh_inspections, column_name="Inspection ID", map_name="map.html")

In [13]:
inspections.Results.unique()

array(['Pass', 'Pass w/ Conditions', 'Out of Business', 'Fail',
       'No Entry', 'Not Ready', 'Business Not Located'], dtype=object)

In [26]:
def get_percentage_results(data, neigh_key="pri_neigh", results="Results", key="Inspection ID", result_type=None):
    neigh_inspections = inspections.groupby(neigh_key).count()[key]
    neigh_results = inspections.groupby([neigh_key, results]).count()[key]
    percentage_results = (neigh_results.div(neigh_inspections) * 100).reset_index()
    if result_type is None:
        return percentage_results
    else:
        return percentage_results[percentage_results.Results == result_type]

In [45]:
percentage_fail = get_percentage_results(inspections, result_type="Fail")
percentage_fail

,pri_neigh,Results,Inspection ID
0,Albany Park,Fail,19.493745
6,Andersonville,Fail,20.370370
13,Archer Heights,Fail,19.052320
19,Armour Square,Fail,18.000000
25,Ashburn,Fail,14.098565
...,...,...,...
581,West Ridge,Fail,22.508306
587,West Town,Fail,17.530390
594,Wicker Park,Fail,19.962802
600,Woodlawn,Fail,27.403846


In [46]:
plot_map(geo_df, data=percentage_fail, column_name="Inspection ID", map_name="map.html")

### Gotta analyze the different types of results. Risks can also be interesting. 
### Below the case of Subway

In [47]:
def get_inspections_restaurant(inspections, restaurant):
    return inspections[inspections["DBA Name"].str.contains(restaurant)]

### Case for subway
inspections_subway = get_inspections_restaurant(inspections, "SUBWAY")
inspections_subway

,Inspection ID,DBA Name,AKA Name,License #,Facility Type,Risk,Address,City,State,Zip,...,Longitude,Location,Historical Wards 2003-2015,Zip Codes,Community Areas,Census Tracts,Wards,date,pri_neigh,sec_neigh
96,2316054,SUBWAY,SUBWAY,2590201.0,Restaurant,Risk 1 (High),2512 W NORTH AVE,CHICAGO,IL,60647.0,...,-87.690207,"{'longitude': '41.910398978211525', 'latitude'...",NaN,NaN,NaN,NaN,NaN,2019-10-18,Wicker Park,"WICKER PARK,WEST TOWN"
180,2315927,SUBWAY,SUBWAY,2590201.0,Restaurant,Risk 1 (High),2512 W NORTH AVE,CHICAGO,IL,60647.0,...,-87.690207,"{'longitude': '41.910398978211525', 'latitude'...",NaN,NaN,NaN,NaN,NaN,2019-10-16,Wicker Park,"WICKER PARK,WEST TOWN"
189,2315950,SUBWAY RESTAURANT,SUBWAY RESTAURANT,1953503.0,Restaurant,Risk 1 (High),3207 W DIVERSEY AVE,CHICAGO,IL,60647.0,...,-87.707742,"{'longitude': '41.931921464989244', 'latitude'...",NaN,NaN,NaN,NaN,NaN,2019-10-16,Logan Square,LOGAN SQUARE
283,2315770,SUBWAY,SUBWAY,2163012.0,Restaurant,Risk 1 (High),4157 W Peterson AVE,CHICAGO,IL,60646.0,...,-87.733433,"{'longitude': '41.98996531393315', 'latitude':...",NaN,NaN,NaN,NaN,NaN,2019-10-11,"Sauganash,Forest Glen","SAUGANASH,FOREST GLEN"
405,2315566,SUBWAY,SUBWAY,2134794.0,Restaurant,Risk 1 (High),6072-6074 N NORTHWEST HWY,CHICAGO,IL,60631.0,...,-87.798097,"{'longitude': '41.99165185328709', 'latitude':...",NaN,NaN,NaN,NaN,NaN,2019-10-09,Norwood Park,NORWOOD PARK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194350,67826,SUBWAY,SUBWAY,1979303.0,Restaurant,Risk 1 (High),4032 W 26TH ST,CHICAGO,IL,60623.0,...,-87.726033,"{'longitude': '41.84434188397153', 'latitude':...",NaN,NaN,NaN,NaN,NaN,2010-01-11,Little Village,LITTLE VILLAGE
194438,58233,SUBWAY,SUBWAY,28635.0,Restaurant,Risk 1 (High),1129 N WESTERN AVE,CHICAGO,IL,60622.0,...,-87.686858,"{'longitude': '41.90204390855024', 'latitude':...",NaN,NaN,NaN,NaN,NaN,2010-01-08,Ukrainian Village,UKRAINIAN VILLAGE AND EAST VILLAGE
194485,67787,SUBWAY STORE # 25458,SUBWAY,1492724.0,Restaurant,Risk 1 (High),1818 W MONTROSE AVE,CHICAGO,IL,60613.0,...,-87.675362,"{'longitude': '41.96161297059614', 'latitude':...",NaN,NaN,NaN,NaN,NaN,2010-01-07,Lincoln Square,LINCOLN SQUARE
194553,154221,SUBWAY SANDWICHES AND SALADS,SUBWAY,2013762.0,Restaurant,Risk 2 (Medium),1513 W fullerton AVE,CHICAGO,IL,60614.0,...,-87.666334,"{'longitude': '41.92506665893317', 'latitude':...",NaN,NaN,NaN,NaN,NaN,2010-01-06,Lincoln Park,LINCOLN PARK


In [48]:
percentage_fail_subway = get_percentage_results(inspections, result_type="Fail")
percentage_fail_subway

,pri_neigh,Results,Inspection ID
0,Albany Park,Fail,19.493745
6,Andersonville,Fail,20.370370
13,Archer Heights,Fail,19.052320
19,Armour Square,Fail,18.000000
25,Ashburn,Fail,14.098565
...,...,...,...
581,West Ridge,Fail,22.508306
587,West Town,Fail,17.530390
594,Wicker Park,Fail,19.962802
600,Woodlawn,Fail,27.403846


In [40]:
plot_map(geo_df, data=percentage_fail_subway, column_name="Inspection ID", map_name="map.html")
## There are a few areas with more fails and it seems like it is in the black part 

## Dealing with Income - for some reason geolocator is not working

In [84]:
income_df = pd.read_csv("data/economic-info.csv")
income_df = income_df.merge(geo_df, left_on="COMMUNITY AREA NAME", right_on="pri_neigh")
income_df.head()

,Community Area Number,COMMUNITY AREA NAME,PERCENT OF HOUSING CROWDED,PERCENT HOUSEHOLDS BELOW POVERTY,PERCENT AGED 16+ UNEMPLOYED,PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA,PERCENT AGED UNDER 18 OR OVER 64,PER CAPITA INCOME,HARDSHIP INDEX,pri_neigh,sec_neigh,shape_area,shape_len,geometry
0,1.0,Rogers Park,7.7,23.6,8.7,18.2,27.5,23939,39.0,Rogers Park,ROGERS PARK,51259902.4506,34052.397576,(POLYGON ((-87.65455590024236 41.9981661505395...
1,2.0,West Ridge,7.8,17.2,8.8,20.8,38.5,23040,46.0,West Ridge,WEST RIDGE,98429094.8621,43020.689458,(POLYGON ((-87.68465309464752 42.0194847735364...
2,3.0,Uptown,3.8,24.0,8.9,11.8,22.2,35787,20.0,Uptown,UPTOWN,65095642.836,46972.790182,(POLYGON ((-87.67397665771156 41.9615254357625...
3,4.0,Lincoln Square,3.4,10.9,8.2,13.4,25.5,37524,17.0,Lincoln Square,LINCOLN SQUARE,71352328.2399,36624.603085,"(POLYGON ((-87.6744075677953 41.9761034052494,..."
4,5.0,North Center,0.3,7.5,5.2,4.5,26.2,57123,6.0,North Center,NORTH CENTER,57054167.85,31391.669754,(POLYGON ((-87.67336415408506 41.9323427454775...


In [7]:
len(income_df), len(geo_df) ## not same size, there are more in the map

(65, 98)

In [8]:
plot_map(geo_df, data=income_df, column_name="PER CAPITA INCOME ", map_name="map.html")

In [9]:
missing_neighs = list(set(geo_df.pri_neigh) - set(income_df["pri_neigh"]))
missing_neighs[:5], len(missing_neighs)

(['Magnificent Mile',
  'Boystown',
  'Jackson Park',
  'Mckinley Park',
  'Montclare'],
 33)

In [51]:
set(income_df["pri_neigh"]) - set(geo_df.pri_neigh)

set()

In [52]:
## lets find the nearest one and assign the same data
geolocator = Nominatim(user_agent="NKPDF", timeout=3)
neigh_point_dict = []
for neigh in missing_neighs:
    location = geolocator.geocode("{} Chicago".format(neigh))
    if location is not None: ## gotta fix this
        neigh_point_dict.append({"pri_neigh": neigh, "geometry": Point(location.longitude, location.latitude)})
missing_neighs_df = gpd.GeoDataFrame(neigh_point_dict, crs={'init': 'epsg:4326'})
missing_neighs_df.head()

####### THERE IS "RUSH & DIVISION" & another one WHICH IS NULL, GOTTA FIX
####### e.g. location = geolocator.geocode("Rush & Division, Chicago".format(neigh)) == None

,pri_neigh,geometry
0,Magnificent Mile,POINT (-87.624228 41.8945229)
1,Boystown,POINT (-87.6492669 41.9438833)
2,Jackson Park,POINT (-87.5804432439724 41.78323175)
3,Mckinley Park,POINT (-87.6736638 41.8316997)
4,Montclare,POINT (-87.8008931 41.9253091)


In [86]:
##pri_neigh is the one we use to plot (from geo_df), data_neigh is the one from income_df
tmp_income_df = gpd.GeoDataFrame(income_df.copy())
missing_neighs_df["data_neigh"] = missing_neighs_df.copy().geometry.apply(get_nearest_neigh, geo_df=tmp_income_df)\
                                    .rename(columns={"geometry": "data_neigh"})
missing_neighs_df.head()

,pri_neigh,geometry,data_neigh
0,Magnificent Mile,POINT (-87.624228 41.8945229),Loop
1,Boystown,POINT (-87.6492669 41.9438833),Lake View
2,Jackson Park,POINT (-87.5804432439724 41.78323175),Woodlawn
3,Mckinley Park,POINT (-87.6736638 41.8316997),Lower West Side
4,Montclare,POINT (-87.8008931 41.9253091),Belmont Cragin


In [87]:
income_df_join = income_df.copy().rename(columns={"pri_neigh": "data_neigh"}).drop(columns="geometry")
missing_neighs_df = missing_neighs_df.merge(income_df_join, how="left", on="data_neigh")
missing_neighs_df.head()

,pri_neigh,geometry,data_neigh,Community Area Number,COMMUNITY AREA NAME,PERCENT OF HOUSING CROWDED,PERCENT HOUSEHOLDS BELOW POVERTY,PERCENT AGED 16+ UNEMPLOYED,PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA,PERCENT AGED UNDER 18 OR OVER 64,PER CAPITA INCOME,HARDSHIP INDEX,sec_neigh,shape_area,shape_len
0,Magnificent Mile,POINT (-87.624228 41.8945229),Loop,32.0,Loop,1.5,14.7,5.7,3.1,13.5,65526,3.0,LOOP,31485185.0318,52640.907563
1,Boystown,POINT (-87.6492669 41.9438833),Lake View,6.0,Lake View,1.1,11.4,4.7,2.6,17.0,60058,5.0,LAKE VIEW,76561177.2562,69715.29313
2,Jackson Park,POINT (-87.5804432439724 41.78323175),Woodlawn,42.0,Woodlawn,2.9,30.7,23.4,16.5,36.1,18672,58.0,WOODLAWN,40515739.083,28960.059037
3,Mckinley Park,POINT (-87.6736638 41.8316997),Lower West Side,31.0,Lower West Side,9.6,25.8,15.8,40.7,32.6,16444,76.0,LOWER WEST SIDE,81550723.8231,43229.368807
4,Montclare,POINT (-87.8008931 41.9253091),Belmont Cragin,19.0,Belmont Cragin,10.8,18.7,14.6,37.3,37.3,15461,70.0,"BELMONT CRAGIN,HERMOSA",109099407.211,43311.706886


In [88]:
#save_missing_neighs = missing_neighs_df.copy()
missing_neighs_df = save_missing_neighs.copy()
missing_neighs_df.head()

In [89]:
income_df = income_df.append(missing_neighs_df.drop(columns="data_neigh"), sort=False)
income_df.head()

C:\Users\Gabriel\Miniconda3\envs\ada\lib\site-packages\pandas\core\frame.py:7123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


,COMMUNITY AREA NAME,Community Area Number,HARDSHIP INDEX,PER CAPITA INCOME,PERCENT AGED 16+ UNEMPLOYED,PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA,PERCENT AGED UNDER 18 OR OVER 64,PERCENT HOUSEHOLDS BELOW POVERTY,PERCENT OF HOUSING CROWDED,geometry,pri_neigh,sec_neigh,shape_area,shape_len
0,Rogers Park,1.0,39.0,23939,8.7,18.2,27.5,23.6,7.7,(POLYGON ((-87.65455590024236 41.9981661505395...,Rogers Park,ROGERS PARK,51259902.4506,34052.397576
1,West Ridge,2.0,46.0,23040,8.8,20.8,38.5,17.2,7.8,(POLYGON ((-87.68465309464752 42.0194847735364...,West Ridge,WEST RIDGE,98429094.8621,43020.689458
2,Uptown,3.0,20.0,35787,8.9,11.8,22.2,24.0,3.8,(POLYGON ((-87.67397665771156 41.9615254357625...,Uptown,UPTOWN,65095642.836,46972.790182
3,Lincoln Square,4.0,17.0,37524,8.2,13.4,25.5,10.9,3.4,"(POLYGON ((-87.6744075677953 41.9761034052494,...",Lincoln Square,LINCOLN SQUARE,71352328.2399,36624.603085
4,North Center,5.0,6.0,57123,5.2,4.5,26.2,7.5,0.3,(POLYGON ((-87.67336415408506 41.9323427454775...,North Center,NORTH CENTER,57054167.85,31391.669754


In [93]:
plot_map(geo_df, data=income_df, column_name="PER CAPITA INCOME ", map_name="map.html")

### This is far from perfect, there are two regions missing and an area that looks rich in the middle of poor neighbourhoods.

In [151]:
park = geo_df[geo_df.pri_neigh == "North Park"]
park

,pri_neigh,sec_neigh,shape_area,shape_len,geometry
51,North Park,"NORTH PARK,ALBANY PARK",70288706.4343,41581.948654,(POLYGON ((-87.70690177438166 41.9830804673769...


In [153]:
geo_df.geometry.distance(park.centroid.values[0]).nsmallest(3)

51    0.000000
52    0.007795
50    0.010500
dtype: float64

### Maybe do like above? Get the smallest and average them?